In [1]:
import json
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from config import Config

c:\Users\HP\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
class DataIngestor:
    def __init__(self, qdrant_client):
        self.client = qdrant_client
        print(f"⏳ Đang tải model {Config.EMBEDDING_MODEL}...")
        self.encoder = SentenceTransformer(Config.EMBEDDING_MODEL)
        print("✅ Model đã sẵn sàng!\n")
    
    def load_menu(self, path):
        """Load dữ liệu info và menu món ăn của nhà hàng"""
        try:
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            print(f"👀 [Giai đoạn 1 - Đọc File] Đã đọc menu.json: Tìm thấy {len(data)} danh mục.")
            docs = []
            
            # 1. Xử lý thông tin chung nhà hàng (Restaurant Info)
            info = data['restaurant']
            info_text = (
                f"Thông tin nhà hàng {info['name']} ({info['name_en']}). "
                f"Địa chỉ: {info['contact']['address']}. "
                f"Giờ mở cửa: {info['business_hours']['display']}. "
                f"SĐT: {info['contact']['phone']}. "
                f"Mô tả: {info['description']}."
            )
            docs.append({"text": info_text, "source": "info", "id": "rest_info"})
            
            # 2. Xử lý Menu (Quan trọng: Ghép description và tags vào text)
            for category in data['menu']['categories']:
                cat_name = category['name_vn']
                for item in category['items']:
                    # Tạo chuỗi văn bản giàu ngữ nghĩa cho Vector Embedding
                    # Kỹ thuật: Ghép hết các trường quan trọng vào 1 đoạn văn
                    content = (
                        f"Món: {item['name_vn']} ({item['name_en']}). "
                        f"Loại: {cat_name}. "
                        f"Giá: {item['price']} VND. "
                        f"Mô tả hương vị: {item.get('description', '')}. " # Lấy mô tả từ V2
                        f"Đặc điểm: {', '.join(item.get('tags', []))}. "   # Lấy tags (spicy, seafood...)
                        f"Thời gian chuẩn bị: {item.get('preparation_time', 0)} phút."
                    )
                    
                    # Lưu metadata để sau này filter nếu cần
                    metadata = {
                        "text": content,
                        "source": "menu",
                        "id": item['id'],
                        "price": item['price'],
                        "category": cat_name,
                        "tags": item.get('tags', [])
                    }
                    docs.append(metadata)

            # 3. Xử lý FAQ (Common Questions)
            for idx, qa in enumerate(data.get('common_questions', [])):
                qa_text = f"Hỏi: {qa['question']} - Trả lời: {qa['answer']}"
                docs.append({"text": qa_text, "source": "faq", "id": f"faq_{idx}"})
                
            return docs

        except FileNotFoundError:
            print("Error: Menu file not found.")
            return []

    def ingest(self):
        """Nạp dữ liệu vào Vector DB"""
        print("--- Bắt đầu nạp dữ liệu vào Vector DB ---")
        menu_data = self.load_menu(os.path.join(Config.DATA_DIR, "menu_v2.json"))
        print(os.path.join(Config.DATA_DIR, "menu_v2.json"))
        print(Config.BASE_DIR)
        
        if not menu_data: 
            print("Không có dữ liệu để nạp.")
            return
        
        # Tạo collection trong Qdrant
        if self.client.collection_exists(collection_name=Config.COLLECTION_NAME):
            self.client.delete_collection(collection_name=Config.COLLECTION_NAME)
            
        self.client.create_collection(
            collection_name=Config.COLLECTION_NAME,
            vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
        )
        print(f"\n⚡ [Giai đoạn 3 - Vector hóa] Đang biến đổi {len(menu_data)} đoạn văn thành số...")
        
        # Vector hóa và upload
        points = []
        embeddings = self.encoder.encode([d['text'] for d in menu_data])
        print(f"   👉 Vector mẫu (3 số đầu/384): {embeddings[0]} ... {embeddings.shape}")
        # Loop tạo PointStruct (kết hợp ID + Vector + Payload)
        for i, doc in enumerate(menu_data):
            points.append(PointStruct(
                id=i,
                vector=embeddings[i].tolist(),
                payload=doc
            ))
        
        print(f"\n📦 [Giai đoạn 4 - Đóng gói] Chuẩn bị đẩy {len(points)} Points vào Qdrant {points}.")
        # Client.upsert -> Đẩy lên Qdrant
        self.client.upsert(
            collection_name=Config.COLLECTION_NAME,
            points=points
        )
        print(f"--- Đã nạp {len(points)} documents vào Qdrant ---")

In [3]:
if __name__ == "__main__":
    # Dùng Qdrant chạy trên RAM (không cần cài Docker để test)
    Client = QdrantClient(":memory:")
    
    ingestor = DataIngestor(Client)
    ingestor.ingest()
    
    # Test thử tìm kiếm để chứng minh dữ liệu đã vào
    print("\n--- 🔎 TEST TÌM KIẾM ---")
    query = "Tôi muốn ăn món gì đó cay cay"
    print(f"Câu hỏi: {query}")
    
    model = SentenceTransformer(Config.EMBEDDING_MODEL)
    hits = Client.search(
        collection_name=Config.COLLECTION_NAME,
        query_vector=model.encode(query).tolist(),
        limit=2
    )
    
    for hit in hits:
        print(f"Kết quả tìm thấy: {hit.payload['text']}")
    

⏳ Đang tải model AITeamVN/Vietnamese_Embedding...
✅ Model đã sẵn sàng!

--- Bắt đầu nạp dữ liệu vào Vector DB ---
👀 [Giai đoạn 1 - Đọc File] Đã đọc menu.json: Tìm thấy 4 danh mục.
d:\2_Workspace\4_Nam_4\1_Hoc_ky_251\4_NLP\4_Project\Hoa-Vien-Restaurant-Chat-App\python\data\menu_v2.json
d:\2_Workspace\4_Nam_4\1_Hoc_ky_251\4_NLP\4_Project\Hoa-Vien-Restaurant-Chat-App\python\src

⚡ [Giai đoạn 3 - Vector hóa] Đang biến đổi 27 đoạn văn thành số...
   👉 Vector mẫu (3 số đầu/384): [ 0.03455431 -0.00976736 -0.02488177 ...  0.0340477  -0.02233206
 -0.00674082] ... (27, 1024)

📦 [Giai đoạn 4 - Đóng gói] Chuẩn bị đẩy 27 Points vào Qdrant [PointStruct(id=0, vector=[0.03455431014299393, -0.009767363779246807, -0.024881767109036446, 0.012239749543368816, -0.06192198395729065, 0.060047343373298645, -0.005188875366002321, 0.010136207565665245, -0.0027340776287019253, -0.022432154044508934, 0.00784455705434084, -0.005580273922532797, -0.0069262078031897545, 0.0001522206439403817, 0.04348893091082573, 0.